# Line Enhancement Basic Flow

* Author: docai-incubator@google.com

## Disclaimer

This tool is not supported by the Google engineering team or product team. It is provided and supported on a best-effort basis by the **DocAI Incubator Team**. No guarantees of performance are implied.

## Objective

This tool built using Python programming language. It converts tables present in pdf to csv files and stores them in GCS bucket by enhancing input pdf files.

# Pre-requisites

This tool requires the following services:

 * Vertex AI Notebook instance
 * Access to Document AI Form Parser & CDE Processor
 * GCS Bucket containing input PDFs & to store output results
 
Google Jupyter Notebook is used for running the python notebook file. Cloud Storage Buckets is needed to store and pass input files to this script & to store results.

CDE for Headers, Create a Custom Document Extractor(CDE) Processor & Configure HITL to review poor performing documents. Train your CDE as per your use-case table by annotating **row headers** & **column headers** for specific use-case-table
* Input for this step is GCS bucket containing re-builted PDF files(which have only tables), now run `batch_process_documents`
* Output JSON files will be stored in GCS bucket 

Sample image after training CDE processor for row columns & header columns
<table>
  <tr>
      <td><b>CDE Sample</b></td>
    <td><img src="./Images/cde_train_sample.png" width=500 height=200></td>
  </tr>
</table> 
Here are sample row headers and column headers which we followed while training CDE for our specific use-case table  

**column headers** are as follow a ["SCC", "DNSH", "DNSH_P", "code", "business_measure", "DNSH_BE", "DNSH_CCA", "DNSH_CCM", "DNSH_CE", "DNSH_WMR", "min_safeguards", "proportion_of_bm", "SCC_BE", "SCC_CCA", "SCC_CCM", "SCC_CE", "SCC_P", "SCC_WMR"] and **row headers** are as follow ["taxonomy_disclosure", "activity"]

# Script

# 1. Import Modules/Packages

**Note** : Please download the **tool_helper_functions.py** Python file before proceeding to further steps.

In [ ]:
import datetime
from tool_helper_functions import (
    batch_process_documents,
    poll_hitl_operations,
    enhance_and_save_pdfs,
    get_processor_metadata,
    parse_document_tables,
)

# 2. Input Details : Configure below Input variables

* **project_id**: GCP project ID
* **project_num**: GCP project Number
* **location**: Processor location `us` or `eu`
* **cde_processor_id**: CDE processor ID to call batch process
* **gcs_input_uri**: GCS folder which contains input pdf files(files with only specific-use-case tables)
* **input_mime_type**: Mime type of input files which is `application/pdf` here
* **gcs_output_bucket_uri**: GCS output bucket uri without trailing slash
* **gcs_output_uri_prefix**: GCS output folder path to store results
* **field_mask**:  To store specific keys of document proto (entities,pages.pageNumber)
* **timeout**: to wait for batch process LRO operation to complete
* **gcs_cde_hitl_output_prefix**: GCS folder which stored HITL output results fro CDE prpcessor
* **line_enhancement_vertical_offset**: Offset used to adjust the placement of the vertical lines, it can be tuned based on the layout
* **line_enhancement_horizontal_offset**: Offset used to adjust the placement of the horizontal lines, it can be tuned based on the layout
* **flow**: for this notebook file flow is `line_enhancement_basic`
* **fp_processor_id**: FP Processor ID to call batch process
* **fp_processor_v1**: FP version ID to call batch process
* **gcs_line_enhance_output_prefix**: GCS prefix to store Line enhancement results
* **gcs_fpoutput_uri_prefix**: GCS prefix to store FP results

In [ ]:
datetime_suffix = "{date:%Y-%m-%d_%H:%M:%S}".format(date=datetime.datetime.now())

project_id = "<your-project-id>"
project_num = "<your-project-number>"
location = "<processor-location>"  # us or eu
cde_processor_id = "<cde-processor-id>"
gcs_input_uri = f"gs://bucket_name/prefix/to_input/{datetime_suffix}"
input_mime_type = "<mime-type-of-input-file>"  # "application/pdf"
gcs_output_bucket_uri = "gs://bucket_name"
gcs_output_uri_prefix = f"output_folder/prefix/{datetime_suffix}"
field_mask = None  # "entities,pages.pageNumber"
timeout = 5000
gcs_cde_hitl_output_prefix = "cde-hitl/output/prefix"
line_enhancement_vertical_offset = 25
line_enhancement_horizontal_offset = 5

flow = "line_enhancement_basic"
fp_processor_id = "<form-processor-id>"
fp_processor_v1 = "<fp-version-id>"
gcs_line_enhance_output_prefix = "line_enhancement_basic/output/prefix"
gcs_fpoutput_uri_prefix = "fp_output/prefix"


line_enhance_prefix = f"{gcs_line_enhance_output_prefix}/{datetime_suffix}"
gcs_output_bucket = gcs_output_bucket_uri.replace("gs://", "")

# 3. Run below code

Now call `batch_process_documents` function to process all files in input folder(each file contains specific-use-case table only), it results metadata & operation_id of batch process(Long Running Operation)

In [ ]:
cde_metadata, cde_operation = batch_process_documents(
    project_id,
    location,
    cde_processor_id,
    gcs_input_uri,
    input_mime_type,
    gcs_output_bucket_uri,
    gcs_output_uri_prefix,
    field_mask,
    timeout,
)
print("CDE batch process completed")

Now use `get_processor_metadata` function from utils module, it takes batch process metsdata as input and results key-value pairs of filenames & it's prefix and hitl operation-id(if input files triggers hitl)

In [ ]:
cde_input_output_map = get_processor_metadata(cde_metadata)
# cde_input_output_map variable data as below
# {'03_Non-Financial_Corporate_Report_2022_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/0', 'hitl': '12795457638097959002'}, '1962771_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/1', 'hitl': '11860520012484438543'}, '2022_VGT_Group Annual Report_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/5', 'hitl': '2523802694474965110'}, 'DE0007030009-JA-2022-EQ-E-00-pg144_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/3', 'hitl': '14342450698739476592'}, 'DE0007030009-JA-2022-EQ-E-00_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/4', 'hitl': '17242897657994716395'}, 'DE000STRA555-JA-2022-EQ-E-00_extracted.pdf': {'cde': 'msci/TESTING/test_cde_output/2023-11-03_05:45:35/4236894205843634293/2', 'hitl': '2909143051612169782'}}

`poll_hitl_operations` is a waiting function to check & resolve HITL triggered documents

In [ ]:
poll_hitl_operations(project_num, location, cde_input_output_map)

Now run `enhance_and_save_pdfs` function from line_enhancement module. Here we are identifying y-coordinates of row-headers to draw horizontal-lines. This is where actual line enhancement process for CDE output files

In [ ]:
# line_enhance_prefix = f"{gcs_line_enhance_output_prefix}/{datetime_suffix}"
enhance_and_save_pdfs(
    gcs_output_bucket,
    gcs_cde_hitl_output_prefix,
    line_enhance_prefix,
    cde_input_output_map,
    line_enhancement_vertical_offset,
    line_enhancement_horizontal_offset,
)

After enhancing pdf i.e, *drawing/brightening* horizontal and vertical lines of table is as below  
![](./Images/after_line_enhancement_sample.png)

In [ ]:
gcs_line_output_path = (
    f"{gcs_output_bucket_uri}/{gcs_line_enhance_output_prefix}/{datetime_suffix}"
)
fp_processor_v = fp_processor_v1

fp_metadata, fp_operation = batch_process_documents(
    project_id,
    location,
    fp_processor_id,
    gcs_line_output_path,
    input_mime_type,
    gcs_output_bucket_uri,
    f"{gcs_fpoutput_uri_prefix}/{datetime_suffix}",
    field_mask,
    timeout,
    fp_processor_v,
)
print("FP batch process completed")

Now use `parse_document_tables` function from utils package. Results will be stored as CSV files in gcs folder of "{gcs_output_uri_prefix}/{flow}/{datetime_suffix}", inside this folder we have number of csv files based on count of tables in input files and it follows file name as "filename_without_extension/pg{page_number}_tb{table_count}.csv"

In [ ]:
print(flow)
## parse the FP output and store the csv in GCS bucket
parse_document_tables(
    gcs_output_bucket,
    f"{gcs_fpoutput_uri_prefix}/{datetime_suffix}",
    f"{gcs_output_uri_prefix}/{flow}/{datetime_suffix}",
)

Refer below image for sample output folder structure, after running `parse_document_tables` function
![](./Images/leb_folder_sample.png)

# 4. Output Samples

Table Sample from pdf file
![](./Images/line_enhancement_basic_table_img.png)

output sample for one-table which stored as csv files in GCS bucket
![](./Images/line_enhancement_basic_table_img(pg1_tb0.csv).png)